In [1]:
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
schlev_2010 = pd.read_csv('./data_sets/us_census/elsec10.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2011 = pd.read_csv('./data_sets/us_census/elsec11.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2012 = pd.read_csv('./data_sets/us_census/elsec12.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2013 = pd.read_csv('./data_sets/us_census/elsec13.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2014 = pd.read_csv('./data_sets/us_census/elsec14.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2015 = pd.read_csv('./data_sets/us_census/elsec15.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2016 = pd.read_csv('./data_sets/us_census/elsec16.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2017 = pd.read_csv('./data_sets/us_census/elsec17.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
schlev_2018 = pd.read_csv('./data_sets/us_census/elsec18.txt', usecols=['STATE', 'NAME', 'SCHLEV', 'YRDATA'], dtype={'STATE': 'str', 'NAME': 'str', 'SCHLEV': 'str', 'YRDATA': 'int'}, index_col=False, na_values='N', na_filter=False)
states = {}
with open('./data_sets/us_census/states_map.json') as f:
    states = json.load(f)
schlev_and_state = pd.concat([schlev_2010, schlev_2011, schlev_2012, schlev_2013, schlev_2014, schlev_2015, schlev_2016, schlev_2017, schlev_2018])
schlev_and_state['STATE'] = schlev_and_state['STATE'].map(lambda x: states[x])

In [3]:
schl_data_2010 = pd.read_csv('./data_sets/us_census/elsec10t.txt', index_col=False)
schl_data_2011 = pd.read_csv('./data_sets/us_census/elsec11t.txt', index_col=False)
schl_data_2012 = pd.read_csv('./data_sets/us_census/elsec12t.txt', index_col=False)
schl_data_2013 = pd.read_csv('./data_sets/us_census/elsec13t.txt', index_col=False)
schl_data_2014 = pd.read_csv('./data_sets/us_census/elsec14t.txt', index_col=False)
schl_data_2015 = pd.read_csv('./data_sets/us_census/elsec15t.txt', index_col=False)
schl_data_2016 = pd.read_csv('./data_sets/us_census/elsec16t.txt', index_col=False)
schl_data_2017 = pd.read_csv('./data_sets/us_census/elsec17t.txt', index_col=False)
schl_data_2018 = pd.read_csv('./data_sets/us_census/elsec18t.txt', index_col=False)
schl_data_2010['YRDATA'] = 10
schl_data_2011['YRDATA'] = 11
schl_data_2012['YRDATA'] = 12
schl_data_2013['YRDATA'] = 13
schl_data_2014['YRDATA'] = 14
schl_data_2015['YRDATA'] = 15
schl_data_2016['YRDATA'] = 16
schl_data_2017['YRDATA'] = 17
schl_data_2018['YRDATA'] = 18
column_names = {}
with open('./data_sets/us_census/column_map.json') as f:
    column_names = json.load(f)
regions = {}
with open('./data_sets/us_census/regions_map.json') as f2:
    regions = json.load(f2)

In [4]:
def map_divisions(state):
    for division, states in regions["divisions"].items():
        if state in states:
            return division
def map_regions(division):
    for region, divisions in regions["regions"].items():
        if division in divisions:
            return region
schlev_data = pd.concat([schl_data_2010, schl_data_2011, schl_data_2012, schl_data_2013, schl_data_2014, schl_data_2015, schl_data_2016, schl_data_2017, schl_data_2018])
schlev_data = pd.merge(schlev_and_state,
                       schlev_data,
                       on=['NAME', 'YRDATA'])
schlev_data = schlev_data[schlev_data['SCHLEV'] == '03']
schlev_data.drop(columns=['SCHLEV', 'IDCENSUS', 'CONUM', 'CSA', 'CBSA', 'NCESID', 'ENROLL'], inplace=True)
schlev_data['DIVISION'] = schlev_data['STATE'].map(map_divisions)
schlev_data['REGION'] = schlev_data['DIVISION'].map(map_regions)
weird_ME_outliers = schlev_data[schlev_data['PPCSTOT'] < 0].index
schlev_data.drop(index=weird_ME_outliers, inplace=True)
schlev_data.head()

,STATE,NAME,YRDATA,TOTALREV,TFEDREV,FEDRCOMP,FEDRSPEC,FEDRNUTR,FEDROTHR,TSTREV,...,PPITOTAL,PPISALWG,PPIEMBEN,PPSTOTAL,PPSPUPIL,PPSSTAFF,PPSGENAD,PPSSCHAD,DIVISION,REGION
0,AL,AUTAUGA COUNTY SCHOOL DISTRICT,10,75102,10038,1850,2857,2289,3042,47735,...,4508.0,2964.0,1190,2333.0,322,259,149,475,East South Central,South
1,AL,BALDWIN COUNTY SCHOOL DISTRICT,10,260945,30989,8795,7272,6023,8899,107281,...,4898.0,3214.0,1251,2972.0,486,419,127,589,East South Central,South
2,AL,BARBOUR COUNTY SCHOOL DISTRICT,10,11335,3158,1350,513,666,629,6269,...,5203.0,3442.0,1361,3719.0,611,352,494,617,East South Central,South
3,AL,EUFAULA CITY SCHOOL DISTRICT,10,26377,4416,1209,990,1075,1142,15624,...,5300.0,3363.0,1364,2725.0,490,277,404,569,East South Central,South
4,AL,BIBB COUNTY SCHOOL DISTRICT,10,33635,5572,1457,1394,1329,1392,20227,...,4709.0,3222.0,1299,2930.0,466,455,278,489,East South Central,South


In [5]:
metadata = pd.read_csv('./data_sets/us_census/mean_household_income/2010_metadata.csv')
column_names = {}
for i in metadata.index:
    row = metadata.iloc[i]
    if row['id'] == 'Mean income (dollars)!!Estimate!!All households':
        column_names[row['GEO_ID']] = 'MEAN INCOME'
    elif row['GEO_ID'] != 'NAME' and row['id'] != 'Mean income (dollars)!!Estimate!!All households' and row['id'] != 'MEAN INCOME':
        column_names[row['GEO_ID']] = row['id']
parent_dir = './data_sets/us_census/mean_household_income/'
tables = [pd.read_csv(f'{parent_dir}2010_data.csv'), pd.read_csv(f'{parent_dir}2011_data.csv'), pd.read_csv(f'{parent_dir}2012_data.csv'), pd.read_csv(f'{parent_dir}2013_data.csv'), pd.read_csv(f'{parent_dir}2014_data.csv'), pd.read_csv(f'{parent_dir}2015_data.csv'), pd.read_csv(f'{parent_dir}2016_data.csv'), pd.read_csv(f'{parent_dir}2017_data.csv'), pd.read_csv(f'{parent_dir}2018_data.csv')]
for i in range(len(tables)):
    table = tables[i]
    table.rename(columns=column_names, inplace=True)
    indices_to_drop = table[table['NAME'].str.contains('Puerto Rico')].index.values
    table.drop(index=indices_to_drop, inplace=True)
    table.drop(index=0, inplace=True)
    table = table.loc[:, ('NAME', 'MEAN INCOME')]
    rplc_str = ' (Bandera, Bexar, and Medina Counties);'
    table['NAME'] = table['NAME'].transform(lambda x: x.replace(rplc_str, ',') if (rplc_str in x) else x)
    table.loc[table['NAME'].str.contains(rplc_str, regex=False)] = table.loc[table['NAME'].str.contains(rplc_str, regex=False)].transform(lambda x: x['NAME'].replace(rplc_str, ','), axis=1)
    table['STATE'] = table['NAME'].transform(lambda x: states[x.split(', ')[1]])
    table['NAME'] = table['NAME'].transform(lambda x: x.split(', ')[0].upper())
    table['YRDATA'] = 10 + i
    table['MEAN INCOME'] = table['MEAN INCOME'].astype(int)
    tables[i] = table
mean_income = pd.concat(tables)
schlev_data = pd.merge(schlev_data,
                       mean_income,
                       on=['NAME', 'STATE', 'YRDATA'])
schlev_data.head()

,STATE,NAME,YRDATA,TOTALREV,TFEDREV,FEDRCOMP,FEDRSPEC,FEDRNUTR,FEDROTHR,TSTREV,...,PPISALWG,PPIEMBEN,PPSTOTAL,PPSPUPIL,PPSSTAFF,PPSGENAD,PPSSCHAD,DIVISION,REGION,MEAN INCOME
0,AL,BALDWIN COUNTY SCHOOL DISTRICT,10,260945,30989,8795,7272,6023,8899,107281,...,3214.0,1251,2972.0,486,419,127,589,East South Central,South,64806
1,FL,CLAY COUNTY SCHOOL DISTRICT,10,45015,2697,672,672,565,788,38799,...,3331.0,1306,2587.0,406,209,352,450,South Atlantic,South,68240
2,FL,CLAY COUNTY SCHOOL DISTRICT,10,311728,39322,5039,10079,6312,17892,172790,...,3646.0,994,2514.0,397,396,80,422,South Atlantic,South,68240
3,AL,CULLMAN COUNTY SCHOOL DISTRICT,10,84449,13214,3403,2906,3091,3814,52612,...,3261.0,1285,2755.0,433,319,113,614,East South Central,South,45295
4,AL,ELMORE COUNTY SCHOOL DISTRICT,10,93950,16195,3076,6214,2693,4212,53822,...,3088.0,1267,2643.0,373,490,174,450,East South Central,South,65957


In [6]:
schlev_data.to_csv(path_or_buf='./data_sets/us_census/clean_district_data.csv', na_rep='NA', index=False)